In [1]:
import cv2
import mediapipe as mp
import math
import numpy as np
from time import sleep
# from ffpyplayer.player import MediaPlayer
# from mediapipe.python.solutions import hands

In [2]:
def dist(x1,y1,x2,y2): # 비교를 위한 거리 계산 함수
    return math.sqrt( math.pow(x1-x2,2) + math.pow(y1-y2,2))

In [3]:
compareIndex = [[18,4],[6,8],[10,12],[14,16],[18,20]] # 마디 index 번호
hand_open = [False,False,False,False,False] # False로 초기화
gesture = [
    [False,True,True,False,False,"Yeah"],
    [True,False,False,False,False,"Good"],
]

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mpHands = mp.solutions.hands
my_hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

In [6]:
#control frame rate
cap = cv2.VideoCapture('../sample_dance/videoplayback.mp4') # jupyter 
# cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

#cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

frame_counter = 0
frameTime = int((1/fps)*1000)  #time of each frame (ms단위, 몇ms당 1frame으로 할지 설정)
print(fps)
extract_time_by_per_frame = 1 #몇프레임 당 한번 측정할지 조절 가능


30.0


In [7]:
# ###start step2 --> 영상 추출
# #extract_file = open('DAIN/extractsamples/result.txt','w')
# cap = cv2.VideoCapture('../sample_dance/videoplayback.mp4') # jupyter 
# # cap = cv2.VideoCapture('./sample_dance/videoplayback.mp4') # vs

# #cap = cv2.VideoCapture('C:/Users/yoond/바탕 화면/00CAPSTONE/Capstone-Design/DAIN/sample_dance/videoplayback.mp4')
# fps = cap.get(cv2.CAP_PROP_FPS)



result = [] # 추출된 영상의 전체 넘파이 배열

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_counter += 1 #increase frame counter

        #Recolor image to RGB
        if ret:
            if frame_counter % extract_time_by_per_frame == 0:
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False

                #Detection
                results = pose.process(image)

                #Recolor image RGB to BGR
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                #Extract landmarks
                try: #success to extract landmarks
                    landmarks = results.pose_landmarks.landmark
                    
                    ret = []
                    for i in range(len(landmarks)):
                        
                        temp = np.array([landmarks[i].x,landmarks[i].y,landmarks[i].z], float)
                        ret = np.append(ret,temp,axis = 0)
                    
                    #print(ret)
                    result = np.append(result,ret,axis=0)
                    #print("extracting\n")
                except:
                    pass
        
        else: #no next frame (end point of video)
            break
            #Show video on screen
        cv2.imshow('dance_file',frame)
        
        if cv2.waitKey(frameTime) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
    #extract_file.close()


print('*****extract success*****')
z = result.shape[0]/(33*3)
result = np.reshape(result,(int(z),33,3)) # frame 수,  tracking좌표 수, xyz 
print(result[0])
print(result.shape)


*****extract success*****
[[ 0.52645749  0.29766154 -0.32991648]
 [ 0.53088862  0.28551757 -0.31765532]
 [ 0.53401184  0.28529289 -0.31763777]
 [ 0.53707582  0.28518003 -0.31765887]
 [ 0.52338666  0.28568935 -0.31538811]
 [ 0.52129871  0.28563076 -0.31544691]
 [ 0.51873541  0.28564876 -0.31548181]
 [ 0.54292494  0.28999984 -0.22596987]
 [ 0.51715505  0.29162216 -0.21588114]
 [ 0.53243446  0.31112784 -0.2942642 ]
 [ 0.52379781  0.31021923 -0.29152724]
 [ 0.56809866  0.36467391 -0.15377082]
 [ 0.49691129  0.36886981 -0.13019724]
 [ 0.58722568  0.45294282 -0.13322279]
 [ 0.49056548  0.46524683 -0.09890861]
 [ 0.57381576  0.53547239 -0.22947505]
 [ 0.50256592  0.55210233 -0.19277146]
 [ 0.56906319  0.56254727 -0.26251683]
 [ 0.50161922  0.57798499 -0.22264425]
 [ 0.56319827  0.5571425  -0.28154853]
 [ 0.50928479  0.57469434 -0.24597403]
 [ 0.56350589  0.54802948 -0.23834935]
 [ 0.51152319  0.5643881  -0.2031918 ]
 [ 0.55474395  0.54785347 -0.01451072]
 [ 0.5160417   0.54985511  0.0144251 ]

In [30]:
# body index
body_index = [
    [11,13,15], # left arm
    [12,14,15], # right arm
    [23,25,27], # left leg
    [24,26,28], # right leg
    
    [14,12,24],
    [13,11,23],
    [23,24,26],
    [24,23,25],
    
]
l1 = result[0][11]
l2 = result[0][13]
l3 = result[0][15]


ll1 = l1-l2
ll2 = l3-l2

innerAB = np.dot(ll1,ll2)
AB = np.linalg.norm(ll1) * np.linalg.norm(ll2)
angle = np.arccos(innerAB/AB)
print(angle) # radian
print(angle/np.pi*180)

1.9262733709641964
110.36733434468643


In [31]:
for num, r in enumerate(result):
    
    for index in body_index:
        l1 = r[index[0]]
        l2 = r[index[1]]
        l3 = r[index[2]]
        
        ll1 = l1-l2
        ll2 = l3-l2
        
        innerAB = np.dot(ll1,ll2)
        AB = np.linalg.norm(ll1) * np.linalg.norm(ll2)
        angle = np.arccos(innerAB/AB)
        print(angle/np.pi*180)
    
    print('\n')
    
    

110.36733434468643
95.54444830520083
135.36930341096675
135.71971185360366
15.655228350493765
15.99525938722047
93.56354154293635
84.91938036651537


130.46302237077305
78.25879234898353
117.99191547519695
55.94215339638443
62.653175059260796
13.67947343003894
127.09298192013634
72.83884715100362


112.3772120541193
123.02744643748528
121.62667705989233
86.03387435084245
24.145468025641467
5.4193831654589415
107.26070381592037
96.80968322771359


157.86737106826163
71.62480793985223
135.8715896305711
139.5040124775253
17.641060707976955
45.444912847170635
99.33904136852507
115.48988122474026


117.66568962651542
14.581194734873652
151.10484657383185
158.2295308963544
100.53186448109541
101.0749225027167
100.0763148068864
85.54879225400305


155.01321422136613
38.31015972769742
127.32550893775532
143.43332658245265
88.99963311991615
33.271461605619606
95.66029487623781
98.45425613490333


141.9259409634316
54.91277199385803
146.80967838469513
110.7430512087998
71.0608091261304
42.532485